# 🔎Unlock the Secrets of ASL: Interactive Dashboard using Plotly!

This notebook presents an interactive dashboard that allows exploring American Sign Language (ASL) landmarks in 2D space. By selecting different ASL categories and IDs, users can filter the displayed landmarks and study the hand shapes and movements associated with specific signs. The dashboard is built with Plotly and Python, and can be used for educational purposes or to support the development of ASL recognition systems. 

With just a few lines of code, the notebook will generate a link to the dashboard, and you can begin exploring any ASL gestures given in the Google - Isolated Sign Language Recognition Dataset. Whether you're a student of ASL or simply interested in exploring this fascinating language, this tool is both easy to use and incredibly informative. The intuitive interface makes it easy to explore the intricacies of ASL gestures, gain a deeper understanding of this rich language, and of course help you fine tune the model by identifying what caused the drop in accuracy for a certain category.

**Multi-page dashboard:**
1. **2D Animation**
2. **Mean Frames**

SNEAK PEAK (🚨Watch All the Recording⚠️):

**You only have to run the notebook and click on the second link at the end of the notebook!**

In [1]:
from IPython.display import HTML
HTML('<div align="center"><iframe align="middle" width="1000" height="904" src="https://www.youtube.com/embed/LIT8zOhnIxs" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" allowfullscreen></iframe></div>')

### Packages

In [2]:
import plotly.io as pio
pio.templates.default = "simple_white"
import plotly.graph_objects as go
!pip install -q jupyter_dash
!pip install dash-cytoscape -q
import dash_cytoscape as cyto 

try:
    from pyngrok import ngrok
except:
    !pip install -q pyngrok
    from pyngrok import ngrok
    
try:
    import dash_bootstrap_components as dbc
except:
    !pip install -q dash_bootstrap_components
    import dash_bootstrap_components as dbc

from dash import dcc, dash
import warnings

warnings.filterwarnings("ignore")
from dash import html, dcc
from dash.dependencies import Input, Output
import pandas as pd
from jupyter_dash import JupyterDash 

!ngrok authtoken 2N6MyaoUfK1C0hvwSAajUzgLURx_yREPXyNkRz9Mv13x15Ba
tunnel = ngrok.connect(8050)

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


## Helper Functions

In [3]:
def assign_color(row):
    if row == 'face':
        return 'red'
    elif 'hand' in row:
        return 'dodgerblue'
    else:
        return 'green'


def assign_order(row):
    if row.type == 'face':
        return row.landmark_index + 101
    elif row.type == 'pose':
        return row.landmark_index + 30
    elif row.type == 'left_hand':
        return row.landmark_index + 80
    else:
        return row.landmark_index

def filter_group(group):
    if group[group.type == 'right_hand'].x.isnull().all():
        new_group = group[group.type != 'right_hand']
        if new_group[new_group.type == 'left_hand'].x.isnull().all():
             return pd.DataFrame(columns=group.columns)
        return group
    else:
        new_group = group[group.type != 'left_hand']
        if new_group[new_group.type == 'right_hand'].x.isnull().all():
             return pd.DataFrame(columns=group.columns)


def visualise2d_landmarks(parquet_df):
    connections = [  # right hand
        [0, 1, 2, 3, 4,],
        [0, 5, 6, 7, 8],
        [0, 9, 10, 11, 12],
        [0, 13, 14, 15, 16],
        [0, 17, 18, 19, 20],

        # pose
        [38, 36, 35, 34, 30, 31, 32, 33, 37],
        [40, 39],
        [52, 46, 50, 48, 46, 44, 42, 41, 43, 45, 47, 49, 45, 51],
        [42, 54, 56, 58, 60, 62, 58],
        [41, 53, 55, 57, 59, 61, 57],
        [54, 53],

        # left hand
        [80, 81, 82, 83, 84, ],
        [80, 85, 86, 87, 88],
        [80, 89, 90, 91, 92],
        [80, 93, 94, 95, 96],
        [80, 97, 98, 99, 100], ]


    frames = sorted(set(parquet_df.frame))
    first_frame = min(frames)
    parquet_df['color'] = parquet_df.type.apply(lambda row: assign_color(row))
    parquet_df['plot_order'] = parquet_df.apply(lambda row: assign_order(row), axis=1)
    first_frame_df = parquet_df[parquet_df.frame == first_frame].copy()
    first_frame_df = first_frame_df.sort_values(["plot_order"]).set_index('plot_order')


    frames_l = []
    for frame in frames:
        filtered_df = parquet_df[parquet_df.frame == frame].copy()
        filtered_df = filtered_df.sort_values(["plot_order"]).set_index("plot_order")
        traces = [go.Scatter(
            x=filtered_df['x'],
            y=filtered_df['y'],
            # z=filtered_df['z'],
            mode='markers',
            marker=dict(
                color=filtered_df.color,
                size=9))]

        for i, seg in enumerate(connections):
            trace = go.Scatter(
                    x=filtered_df.loc[seg]['x'],
                    y=filtered_df.loc[seg]['y'],
                    # z=filtered_df.loc[seg]['z'],
                    mode='lines',
            )
            traces.append(trace)
        frame_data = go.Frame(data=traces, traces = [i for i in range(17)])
        frames_l.append(frame_data)

    traces = [go.Scatter(
        x=first_frame_df['x'],
        y=first_frame_df['y'],
        # z=first_frame_df['z'],
        mode='markers',
        marker=dict(
            color=first_frame_df.color,
            size=9
        )
    )]
    for i, seg in enumerate(connections):
        trace = go.Scatter(
            x=first_frame_df.loc[seg]['x'],
            y=first_frame_df.loc[seg]['y'],
            # z=first_frame_df.loc[seg]['z'],
            mode='lines',
            line=dict(
                color='black',
                width=2
            )
        )
        traces.append(trace)
    fig = go.Figure(
        data=traces,
        frames=frames_l
    )

    # Layout
    fig.update_layout(
        title="ASL Sign Visualization",
        width=1000,
        height=1800,
        scene={
            'aspectmode': 'data',
        },
        updatemenus=[
            {
                "buttons": [
                    {
                        "args": [None, {"frame": {"duration": 100,
                                                  "redraw": True},
                                        "fromcurrent": True,
                                        "transition": {"duration": 0}}],
                        "label": "&#9654;",
                        "method": "animate",
                    },

                ],
                "direction": "left",
                "pad": {"r": 100, "t": 100},
                "font": {"size":30},
                "type": "buttons",
                "x": 0.1,
                "y": 0,
            }
        ],
    )
    camera = dict(
        up=dict(x=0, y=-1, z=0),
        eye=dict(x=0, y=0, z=2.5)
    )
    fig.update_layout(scene_camera=camera, showlegend=False)
    fig.update_layout(xaxis = dict(visible=False),
            yaxis = dict(visible=False),
    )
    fig.update_yaxes(autorange="reversed")

    # fig.show()
    return fig

def visualiseavg_landmarks(parquet_df):
    connections = [  # right hand
        [0, 1, 2, 3, 4,],
        [0, 5, 6, 7, 8],
        [0, 9, 10, 11, 12],
        [0, 13, 14, 15, 16],
        [0, 17, 18, 19, 20],

        # pose
        [38, 36, 35, 34, 30, 31, 32, 33, 37],
        [40, 39],
        [52, 46, 50, 48, 46, 44, 42, 41, 43, 45, 47, 49, 45, 51],
        [42, 54, 56, 58, 60, 62, 58],
        [41, 53, 55, 57, 59, 61, 57],
        [54, 53],

        # left hand
        [80, 81, 82, 83, 84, ],
        [80, 85, 86, 87, 88],
        [80, 89, 90, 91, 92],
        [80, 93, 94, 95, 96],
        [80, 97, 98, 99, 100], ]

    first_frame_df = parquet_df.copy()
    first_frame_df['color'] = first_frame_df.type.apply(lambda row: assign_color(row))
    first_frame_df['plot_order'] = first_frame_df.apply(lambda row: assign_order(row), axis=1)
    first_frame_df = first_frame_df.groupby("plot_order").agg({'x':'mean', 'y':'mean', 'color':'first'}).reset_index()
    first_frame_df = first_frame_df.sort_values(["plot_order"]).set_index('plot_order')

    traces = [go.Scatter(
        x=first_frame_df['x'],
        y=first_frame_df['y'],
        # z=first_frame_df['z'],
        mode='markers',
        marker=dict(
            color=first_frame_df.color,
            size=9
        )
    )]
    for i, seg in enumerate(connections):
        trace = go.Scatter(
            x=first_frame_df.loc[seg]['x'],
            y=first_frame_df.loc[seg]['y'],
            # z=first_frame_df.loc[seg]['z'],
            mode='lines',
            line=dict(
                color='black',
                width=2
            )
        )
        traces.append(trace)
    fig = go.Figure(
        data=traces,
    )

    # Layout
    fig.update_layout(
        title="ASL Sign Visualization",
        width=1000,
        height=1800)
    fig.update_layout(showlegend=False)
    fig.update_layout(xaxis = dict(visible=False),
                      yaxis = dict(visible=False),
                      )
    fig.update_yaxes(autorange="reversed")

    return fig

## App & Navigation bar

In [4]:
app = JupyterDash(__name__, server_url=tunnel.public_url, meta_tags=[{"name": "viewport", "content": "width=device-width"}],
                external_stylesheets=[dbc.themes.BOOTSTRAP],suppress_callback_exceptions=True)

def Navbar():
    layout = html.Div([
        dbc.NavbarSimple(
            children=[
                dbc.NavItem(dbc.NavLink("2D Visuals", href="/landmark2d", id="landmark2d")),
                dbc.NavItem(dbc.NavLink("Mean Visual", href="/landmarkavg", id="landmarkavg")),
            ] ,
            brand="Landmark 2D Visualization Dashboard",
            brand_href="/landmark2d",
            id="navbar",
            color="dark",
            dark=True,
        ),
    ])

    return layout

data_directory = '/kaggle/input/asl-signs/'
train_data = pd.read_csv(data_directory + 'train.csv')
dropdown_options = train_data.groupby('sign')['sequence_id'].apply(list).to_dict()
names = list(dropdown_options.keys())
nestedOptions = dropdown_options[names[0]]

## 2D Landmarks Video Page

In [5]:
def visual_dash(sequence_id):
    data = train_data.copy()
    parquet_file = data[data.sequence_id == sequence_id]
    parquet_path = parquet_file.path.values[0]
    sign_cat = parquet_file.sign.values[0]

    parquet_df = pd.read_parquet(data_directory + parquet_path)
    fig = visualise2d_landmarks(parquet_df)

    return fig, sign_cat

fig, sign_cat = visual_dash(1002052130)


sign_dropdown1 = dcc.Dropdown(options=[{'label':name, 'value':name} for name in names],
                                  id='sign_dropdown',
                                  clearable=False,
                                  value = names[0], className="dbc",
                                  placeholder='Select a Sign', maxHeight=200)

sequence_dropdown1 = dcc.Dropdown(id='sequence_dropdown',
                               clearable=False,
                               className="dbc",
                               placeholder='Select a Sequence ID', maxHeight=200)

layout_2d = dbc.Container(
    [dbc.Row([dbc.Col(sign_dropdown1),
              dbc.Col(sequence_dropdown1),]),
     dbc.Row([html.H1(id='sign_cat'),
         dbc.Col([
             dcc.Graph(id='fig', figure=fig,
                       style={'height': 2000}),
             html.Hr()
         ], width={'size': 12, 'offset': 0, 'order': 1})])]
)

## Mean Frame Page

In [6]:
def visualmean_dash(sequence_id):
    data = train_data.copy()
    parquet_file = data[data.sequence_id == sequence_id]
    parquet_path = parquet_file.path.values[0]
    sign_cat = parquet_file.sign.values[0]

    parquet_df = pd.read_parquet(data_directory + parquet_path)
    fig = visualiseavg_landmarks(parquet_df)

    return fig, sign_cat

fig, sign_cat = visualmean_dash(1187993400)


sign_dropdown = dcc.Dropdown(options=[{'label':name, 'value':name} for name in names],
                             id='sign_dropdown',
                             clearable=False,
                             value = names[0], className="dbc",
                             placeholder='Select a Sign', maxHeight=200)

sequence_dropdown = dcc.Dropdown(id='sequence_dropdown',
                                 clearable=False,
                                 className="dbc",
                                 placeholder='Select a Sequence ID', maxHeight=200)

layout_mean = dbc.Container(
    [dbc.Row([dbc.Col(sign_dropdown),
              dbc.Col(sequence_dropdown),]),
     dbc.Row([html.H1(id='sign_cat2'),
              dbc.Col([
                  dcc.Graph(id='fig2', figure=fig,
                            style={'height': 2000}),
                  html.Hr()
              ], width={'size': 12, 'offset': 0, 'order': 1})])]
)

## Putting Everything Together!

In [7]:
nav = Navbar()

app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    nav,
    html.Div(id='page-content', children=[]),
])

@app.callback(Output('page-content', 'children'),
              [Input('url', 'pathname')])

def display_page(pathname):
    if pathname == '/landmark2d':
        return layout_2d
    if pathname == '/landmarkavg':
        return layout_mean
    else:
        return layout_2d

@app.callback(
     Output('sequence_dropdown', 'options'),
    [Input('sign_dropdown', 'value')]
)
def update_dropdown(name):
    return [{'label': i, 'value': i} for i in dropdown_options[name]]

@app.callback([Output(component_id="fig", component_property="figure"),
               Output(component_id="sign_cat", component_property="children")],
              [Input('sequence_dropdown', 'value')])
def callback_function(sequence_id):
    fig, sign_cat = visual_dash(sequence_id)
    return fig, sign_cat

@app.callback([Output(component_id="fig2", component_property="figure"),
               Output(component_id="sign_cat2", component_property="children")],
              [Input('sequence_dropdown', 'value')])
def callback_function(sequence_id):
    fig, sign_cat = visualmean_dash(sequence_id)
    return fig, sign_cat

# Run the app on localhost:8050
app.run_server(mode='external')

Dash is running on http://127.0.0.1:8050/

Dash app running on http://683e-35-204-225-11.ngrok.io/
